# SageMaker workshop: detecting fraudulent credit card transactions

Linear models are supervised learning algorithms used for solving either classification or regression problems. For input, you give the model labeled examples (x, y). x is a high-dimensional vector and y is a numeric label. For binary classification problems, the label must be either 0 or 1. For multiclass classification problems, the labels must be from 0 to num_classes - 1. For regression problems, y is a real number. The algorithm learns a linear function, or, for classification problems, a linear threshold function, and maps a vector x to an approximation of the label y.

The Amazon SageMaker linear learner algorithm provides a solution for both classification and regression problems. With the Amazon SageMaker algorithm, you can simultaneously explore different training objectives and choose the best solution from a validation set. You can also explore a large number of models and choose the best. The best model optimizes either of the following:

- Continuous objectives, such as mean square error, cross entropy loss, absolute error.

- Discrete objectives suited for classification, such as F1 measure, precision, recall, or accuracy.

Compared with methods that provide a solution for only continuous objectives, the Amazon SageMaker linear learner algorithm provides a significant increase in speed over naive hyperparameter optimization techniques. It is also more convenient.

## Detecting credit card fraud

In this workshop, we'll look at a credit card fraud detection dataset.  The data set (Dal Pozzolo et al. 2015) was downloaded from [Kaggle](https://www.kaggle.com/mlg-ulb/creditcardfraud/data).  We have features and labels for over a quarter million credit card transactions, each of which is labeled as fraudulent or not fraudulent.  We'd like to train a model based on the features of these transactions so that we can predict risky or fraudulent transactions in the future.  This is a binary classification problem.  

We'll walk through training linear learner with various settings and deploying an inference endpoint.  We'll evaluate the quality of our models by hitting that endpoint with observations from the test set.  We can take the real-time predictions returned by the endpoint and evaluate them against the ground-truth labels in our test set.

Next, we'll apply the linear learner threshold tuning functionality to get better precision without sacrificing recall.  Then, we'll push the precision even higher using the linear learner new class weights feature.  Because fraud can be extremely costly, we would prefer to have high recall, even if this means more false positives.  This is especially true if we are building a first line of defense, flagging potentially fraudulent transactions for further review before taking actions that affect customers.

First we'll do some preprocessing on this data set: we'll shuffle the examples and split them into train and test sets.  To run this under notebook under your own AWS account, you'll need to change the Amazon S3 locations.  First download the raw data from the s3 location below to this instance.  Only 0.17% of the data have positive labels, making this a challenging classification problem.

In [ ]:
!wget https://loftstockholm.s3.eu-central-1.amazonaws.com/creditcard.csv.zip

In [ ]:
import boto3
import io
import matplotlib.pyplot as plt
import numpy as np 
import os
import pandas as pd 

import sagemaker
import sagemaker.amazon.common as smac
from sagemaker import get_execution_role
from sagemaker.predictor import csv_serializer, json_deserializer
from sagemaker.amazon.amazon_estimator import get_image_uri

At this point we'll have to create an S3 bucket where SageMaker will be reading and writing files to and from (training data, model artifacts, checkpoints,...). Go to the s3 service in the AWS console, create a bucket and fill it in below.

In [ ]:
# Set data locations
bucket = 'llworkshop' # TODO: replace this with your own bucket
prefix = 'sagemaker/DEMO-linear-learner-loss-weights'
s3_train_key = '{}/train/recordio-pb-data'.format(prefix)
s3_train_path = os.path.join('s3://', bucket, s3_train_key)
local_raw_data = 'creditcard.csv.zip'
role = get_execution_role()

In [ ]:
# Check access to s3 bucket. Attempt reading items in bucket: this shouldn't raise any errors. 
# You shouldn't get any items if you just created the bucket.
for obj in boto3.resource('s3').Bucket(bucket).objects.all():
    print(obj.key)

In [ ]:
# Read the data, shuffle, and split into train and test sets, separating the labels (last column) from the features
raw_data = pd.read_csv(local_raw_data).to_numpy()
np.random.seed(0)
np.random.shuffle(raw_data)
train_size = int(raw_data.shape[0] * 0.7)
train_features  = raw_data[:train_size, :-1]
train_labels = raw_data[:train_size, -1]
test_features = raw_data[train_size:, :-1]
test_labels = raw_data[train_size:, -1]

# Convert the processed training data to protobuf and write to S3 for linear learner
vectors = np.array([t.tolist() for t in train_features]).astype('float32')
labels = np.array([t.tolist() for t in train_labels]).astype('float32')
buf = io.BytesIO()
smac.write_numpy_to_dense_tensor(buf, vectors, labels)
buf.seek(0)
boto3.resource('s3').Bucket(bucket).Object(s3_train_key).upload_fileobj(buf)

Add a convenience function for evaluating the model.  To make predictions, we need to set up a model hosting endpoint.  Then we feed test features to the endpoint and receive predicted test labels.  To evaluate the models we create in this exercise, we'll capture predicted test labels and compare them to actuals using some common binary classification metrics.

Complete the code below to calculate recall and precision.
See https://en.wikipedia.org/wiki/F1_score

In [ ]:
def evaluate(linear_predictor, test_features, test_labels, model_name, verbose=True):
    """
    Evaluate a model on a test set given the prediction endpoint.  Return binary classification metrics.
    """
    # split the test data set into 100 batches and evaluate using prediction endpoint
    prediction_batches = [linear_predictor.predict(batch)['predictions'] for batch in np.array_split(test_features, 100)]
    # parse raw predictions json to exctract predicted label
    test_preds = np.concatenate([np.array([x['predicted_label'] for x in batch]) for batch in prediction_batches])
    
    # calculate true positives, false positives, true negatives, false negatives
    tp = np.logical_and(test_labels, test_preds).sum()
    fp = np.logical_and(1-test_labels, test_preds).sum()
    tn = np.logical_and(1-test_labels, 1-test_preds).sum()
    fn = np.logical_and(test_labels, 1-test_preds).sum()
    
    # calculate binary classification metrics
    recall = tp / (tp + fn) # TODO
    precision = tp / (tp + fp) # TODO
    accuracy = (tp + tn) / (tp + fp + tn + fn)
    f1 = 2 * precision * recall / (precision + recall)
    
    if verbose:
        print(pd.crosstab(test_labels, test_preds, rownames=['actuals'], colnames=['predictions']))
        print("\n{:<11} {:.3f}".format('Recall:', recall))
        print("{:<11} {:.3f}".format('Precision:', precision))
        print("{:<11} {:.3f}".format('Accuracy:', accuracy))
        print("{:<11} {:.3f}".format('F1:', f1))
        
    return {'TP': tp, 'FP': fp, 'FN': fn, 'TN': tn, 'Precision': precision, 'Recall': recall, 'Accuracy': accuracy, 
             'F1': f1, 'Model': model_name}

And finally we'll add a convenience function to delete prediction endpoints after we're done with them:

In [ ]:
def delete_endpoint(predictor):
        try:
            boto3.client('sagemaker').delete_endpoint(EndpointName=predictor.endpoint)
            print('Deleted {}'.format(predictor.endpoint))
        except:
            print('Already deleted: {}'.format(predictor.endpoint))

Sagemaker provides containers for the built-in algorithms in each region.  
You can find the image locations here: https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-algo-docker-registry-paths.html .  
Luckily there's a helper function that does that for us:

In [ ]:
container = get_image_uri(boto3.Session().region_name, 'linear-learner')
print('Container image for this region: {}'.format(container))

Let's begin by training a binary classifier model with the linear learner default settings.  Note that we're setting the number of epochs to 40, which is much higher than the default of 15 epochs.  With early stopping, we don't have to worry about setting the number of epochs too high.  Linear learner will stop training automatically after the model has converged.

Have a look at https://docs.aws.amazon.com/sagemaker/latest/dg/ll_hyperparameters.html to figure out what keys to use in the hyperparams dictionary.

The documentation for the Estimator object is here https://sagemaker.readthedocs.io/en/stable/api/training/estimators.html#sagemaker.estimator.Estimator

Tip: see get_execution_role() helper function above which returns this instance's role; you'll need it below.

In [ ]:
# Training a binary classifier with default settings: logistic regression
# TODO: construct the hyperparam dict
# - 40 epochs
# - set the correct predictor type

# TODO: construct the estimator
# train on 1 ml.c5.4xlarge instance

defaults_est = sagemaker.estimator.Estimator(
    container,
    role,
    train_instance_count=1,
    train_instance_type='ml.c5.4xlarge',
    output_path='s3://{}/{}/defaults/output'.format(bucket, prefix),
    hyperparameters={
        'predictor_type': 'binary_classifier',
        'epochs': 40
    }
)

Train the model. This will take a few minutes.  
See: https://sagemaker.readthedocs.io/en/stable/api/training/estimators.html#sagemaker.estimator.EstimatorBase.fit

In [ ]:
# TODO: provide path to the training data
defaults_est.fit({'train': s3_train_path})

How many epochs did we train for? Why not 40?

Deploy an endpoint. This will again take several minutes.  
See https://sagemaker.readthedocs.io/en/stable/overview.html#using-estimators

In [ ]:
# TODO: deploy an estimator on 1 ml.m4.xlarge instance
defaults_predictor = defaults_est.deploy(
    initial_instance_count=1, 
    instance_type='ml.m4.xlarge'
)
defaults_predictor.content_type = 'text/csv'
defaults_predictor.serializer = csv_serializer
defaults_predictor.deserializer = json_deserializer

Use our evaluate() helper function to evaluate the defaults_predictor we just deployed.

In [ ]:
evaluate(defaults_predictor, test_features, test_labels, 'Default')

And now we'll produce a model with a threshold tuned for the best possible precision with recall fixed at 90%:

In [ ]:
# Training a binary classifier with automated threshold tuning
# TODO: construct the hyper params dict
# binary classifier
# set precision at target recall as the model selection criteria
# set target recall to 0.9
# set epochs to 40

autothresh_est = sagemaker.estimator.Estimator(
    container,
    role,
    train_instance_count=1,
    train_instance_type='ml.c5.4xlarge',
    output_path='s3://{}/{}/autothresh/output'.format(bucket, prefix),
    sagemaker_session=sagemaker.Session(),
    hyperparameters={
        'predictor_type': 'binary_classifier',
        'binary_classifier_model_selection_criteria': 'precision_at_target_recall', 
        'target_recall': 0.9,
        'epochs': 40
    }
)

Train the model and deploy an endpoint.

In [ ]:
autothresh_est.fit({'train': s3_train_path})

In [ ]:
# TODO: deploy an estimator on 1 ml.m4.xlarge instance
autothresh_predictor = autothresh_est.deploy(
    initial_instance_count=1, 
    instance_type='ml.m4.xlarge'
)
autothresh_predictor.content_type = 'text/csv'
autothresh_predictor.serializer = csv_serializer
autothresh_predictor.deserializer = json_deserializer

In [ ]:
# TODO: evaluate the deployed endpoint
evaluate(autothresh_predictor, test_features, test_labels, 'AutoTresh', False)

### Managed spot training

EC2 Spot Instances have long been a great cost optimization feature, and spot training is now available on SageMaker.

This blog post has more info: [ https://aws.amazon.com/blogs/aws/managed-spot-training-save-up-to-90-on-your-amazon-sagemaker-training-jobs/ ]

In [ ]:
spot_est = sagemaker.estimator.Estimator(
    container,
    role,
    train_instance_count=1,
    train_instance_type='ml.m4.xlarge',
    output_path='s3://{}/{}/autothresh/output'.format(bucket, prefix),
    sagemaker_session=sagemaker.Session(),
    hyperparameters={
        'feature_dim': 30,
        'predictor_type': 'binary_classifier',
        'binary_classifier_model_selection_criteria': 'precision_at_target_recall', 
        'target_recall': 0.9,
        'epochs': 40
    },
    train_use_spot_instances=True,
    train_max_run=600,
    train_max_wait=3600
)
spot_est.fit({'train': s3_train_path})

Take note of the cost savings shown at the bottom of the output from the fit() call above.

### Bonus: Improving recall with class weights

Now we'll improve on these results using a new feature added to linear learner: class weights for binary classification.  We introduced this feature in the *Class Weights* section, and now we'll look into its application to the credit card fraud dataset by training a new model with balanced class weights:

In [ ]:
# Training a binary classifier with class weights and automated threshold tuning
# TODO: configure the hyperparams so that Linear Learner will use balanced class weights
class_weights_est = sagemaker.estimator.Estimator(
    container,
    role,
    train_instance_count=1,
    train_instance_type='ml.m4.xlarge',
    output_path='s3://{}/{}/class_weights/output'.format(bucket, prefix),
    sagemaker_session=sagemaker.Session(),
    hyperparameters={
        'feature_dim': 30,
        'predictor_type': 'binary_classifier',
        'binary_classifier_model_selection_criteria': 'precision_at_target_recall', 
        'target_recall': 0.9,
        'positive_example_weight_mult': 'balanced',
        'epochs': 40
    }
)

In [ ]:
# TODO: train and deploy the model
class_weights_est.fit({'train': s3_train_path})

class_weights_predictor = class_weights_est.deploy(
    initial_instance_count=1, 
    instance_type='ml.m4.xlarge'
)
class_weights_predictor.content_type = 'text/csv'
class_weights_predictor.serializer = csv_serializer
class_weights_predictor.deserializer = json_deserializer

In [ ]:
# TODO: evaluate the deployed estimator
evaluate(class_weights_predictor, test_features, test_labels, 'ClassWeights', False)

#### Note on target vs. observed recall

It's worth taking some time to look more closely at these results.  If we asked linear learner for a model calibrated to a target recall of 0.9, then why didn't we get exactly 90% recall on the test set?  The reason is the difference between training, validation, and testing.  Linear learner calibrates thresholds for binary classification on the validation data set when one is provided, or else on the training set.  Since we did not provide a validation data set, the threshold were calculated on the training data.  Since the training, validation, and test data sets don't match exactly, the target recall we request is only an approximation.  In this case, the threshold that produced 90% recall on the training data happened to produce only 85-90% recall on the test data (due to some randomness in training, the results will vary from one run to the next).  The variation of recall in the test set versus the training set is dependent on the number of positive points. In this example, although we have over 280,000 examples in the entire dataset, we only have 337 positive examples, hence the large difference.  The accuracy of this approximation can be improved by providing a large validation data set to get a more accurate threshold, and then evaluating on a large test set to get a more accurate benchmark of the model and its threshold.  For even more fine-grained control, we can set the number of calibration samples to a higher number.  It's default value is already quite high at 10 million samples:

    num_calibration_samples=10000000,

#### A note on the SageMaker SDK
Note that there is also this handy LinearLearner class in the SageMaker SDK: https://sagemaker.readthedocs.io/en/stable/algorithms/linear_learner.html

#### Clean Up

Finally we'll clean up by deleting the prediction endpoints we set up:

In [ ]:
for predictor in [defaults_predictor, autothresh_predictor, class_weights_predictor]:
    delete_endpoint(predictor)

We've just shown how to use the linear learner new early stopping feature, new loss functions, and new class weights feature to improve credit card fraud prediction.  Class weights can help you optimize recall or precision for all types of fraud detection, as well as other classification problems with rare events, like ad click prediction or mechanical failure prediction.  Try using class weights in your binary classification problem, or try one of the new loss functions for your regression problems: use quantile prediction to put confidence intervals around your predictions by learning 5% and 95% quantiles.  For more information about new loss functions and class weights, see the linear learner [documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/linear-learner.html).

##### References
Andrea Dal Pozzolo, Olivier Caelen, Reid A. Johnson and Gianluca Bontempi. Calibrating Probability with Undersampling for Unbalanced Classification. In Symposium on Computational Intelligence and Data Mining (CIDM), IEEE, 2015.  See link to full license text on [Kaggle](https://www.kaggle.com/mlg-ulb/creditcardfraud).